In [1]:
import time
from datetime import timedelta

import html
import re

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

import dask
import dask.dataframe as dd
import dask.bag as bd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

### Note
- Some of the files were not complete use the s1_find_incomp_chekstyle_xml.ipnb to find them and remove them
- and the empty ones as well

In [2]:
##########################################################
#Step 1. Format 'pmd_rules_results.xml' from Linux terminal#
##########################################################
# -'pmd_rules_results.xml' is not properly formated
# -We used linux format command to format 'pmd_rules_results.xml' to save it in 'pmd_rules_results_fmt.xml'
# $  xmllint --format checkstyle-result0.xml > checkstyle-result_fmt0.xml
##############################################################################
#Step 2. Remove some lines in 'checkstyle-result_fmt0.xml' from Linux terminal#
##############################################################################
# remove lines 1 to 2 and the last line of the 'checkstyle-result_fmt0.xml'
# MacOS $  sed -i '' '1,2d;$d' checkstyle-result_fmt0.xml 
# Linux  $   sed -i '1,2d;$d' checkstyle-result_fmt0.xml            
# blocksize breaks it into partitions
# Use '</file>\n' as linedelimiter
chk_rules_result_bd = bd.read_text('checkstylexmlreports_xml/checkstyle-result*.xml', linedelimiter='</file>')
#chk_rules_result_bd = bd.read_text('checkstylexmlreports_xml/checkstyle-result0.xml', linedelimiter='</file>\n')
#chk_rules_result_bd = bd.read_text('checkstylexmlreports_xml/checkstyle-result71.xml', linedelimiter='</file>')

In [3]:
# check to see number of partitions
# find the number of partitions
#pmd_rules_result_bg.npartitions # or
chk_rules_result_bd

dask.bag<bag-from-delayed, npartitions=577>

In [4]:
# look at the first row
chk_rules_result_bd.take(1)

('<?xml version="1.0" encoding="UTF-8"?>\n<checkstyle version="9.1">\n<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/CheckStyle1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_30381834_6434168_4179_3.java">\n<error line="1" severity="error" message="File does not end with a newline." source="com.puppycrawl.tools.checkstyle.checks.NewlineAtEndOfFileCheck"/>\n<error line="1" severity="error" message="Missing package-info.java file." source="com.puppycrawl.tools.checkstyle.checks.javadoc.JavadocPackageCheck"/>\n<error line="1" column="1" severity="error" message="Utility classes should not have a public or default constructor." source="com.puppycrawl.tools.checkstyle.checks.design.HideUtilityClassConstructorCheck"/>\n<error line="1" column="14" severity="error" message="Name &apos;Code_30381834_6434168_4179_3&apos; must match pattern &apos;^[A-Z][a-zA-Z0-9]*$&apos;." sour

In [5]:
# look at the first 50 row
#chk_rules_result_bd.take(50)

### We are intrested in getting the 'file'
- so we can get rid of the files
- then filter

In [6]:
#@@#
# replace all the '<xml ...>' header with an empty string so that we can access it using xml.etree.ElementTree
chk_rules_result_bd = chk_rules_result_bd.map(lambda line: re.sub(r'\s*<\s*\?xml.*\?>', '', line))

In [7]:
#@@#
# look at the first row
chk_rules_result_bd.take(1)

('\n<checkstyle version="9.1">\n<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/CheckStyle1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_30381834_6434168_4179_3.java">\n<error line="1" severity="error" message="File does not end with a newline." source="com.puppycrawl.tools.checkstyle.checks.NewlineAtEndOfFileCheck"/>\n<error line="1" severity="error" message="Missing package-info.java file." source="com.puppycrawl.tools.checkstyle.checks.javadoc.JavadocPackageCheck"/>\n<error line="1" column="1" severity="error" message="Utility classes should not have a public or default constructor." source="com.puppycrawl.tools.checkstyle.checks.design.HideUtilityClassConstructorCheck"/>\n<error line="1" column="14" severity="error" message="Name &apos;Code_30381834_6434168_4179_3&apos; must match pattern &apos;^[A-Z][a-zA-Z0-9]*$&apos;." source="com.puppycrawl.tools.checkstyle.ch

In [8]:
#@@#
# replace all the '<checkstyle>' and '</checkstyle>' with empty string so that we can access it using xml.etree.ElementTree
#chk_rules_result_bd = chk_rules_result_bd.map(lambda line: re.sub(r'\n*\s*<\s*checkstyle.*>|.*/checkstyle>\n*', '', line))
chk_rules_result_bd = chk_rules_result_bd.map(lambda line: re.sub(r'\s*<\s*checkstyle.*>|.*/checkstyle>', '', line))

In [9]:
#@@#
# look at the first row
chk_rules_result_bd.take(1)

('\n<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/CheckStyle1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_30381834_6434168_4179_3.java">\n<error line="1" severity="error" message="File does not end with a newline." source="com.puppycrawl.tools.checkstyle.checks.NewlineAtEndOfFileCheck"/>\n<error line="1" severity="error" message="Missing package-info.java file." source="com.puppycrawl.tools.checkstyle.checks.javadoc.JavadocPackageCheck"/>\n<error line="1" column="1" severity="error" message="Utility classes should not have a public or default constructor." source="com.puppycrawl.tools.checkstyle.checks.design.HideUtilityClassConstructorCheck"/>\n<error line="1" column="14" severity="error" message="Name &apos;Code_30381834_6434168_4179_3&apos; must match pattern &apos;^[A-Z][a-zA-Z0-9]*$&apos;." source="com.puppycrawl.tools.checkstyle.checks.naming.TypeNameCheck"/>

In [10]:
# Gets Rid of the empty partitions in Bags
def cull_empty_partitions(bag):
    """
    When bags are created by filtering or grouping from a different bag,
    it retains the original bag's partition count, even if a lot of the
    partitions become empty.
    Those extra partitions add overhead, so it's nice to discard them.
    This function drops the empty partitions.
    """
    bag = bag.persist()
    def get_len(partition):
        # If the bag is the result of bag.filter(),
        # then each partition is actually a 'filter' object,
        # which has no __len__.
        # In that case, we must convert it to a list first.
        if hasattr(partition, '__len__'):
            return len(partition)
        return len(list(partition))
    partition_lengths = bag.map_partitions(get_len).compute()

    # Convert bag partitions into a list of 'delayed' objects
    lengths_and_partitions = zip(partition_lengths, bag.to_delayed())

    # Drop the ones with empty partitions
    partitions = (p for l,p in lengths_and_partitions if l > 0)

    # Convert from list of delayed objects back into a Bag.
    return dask.bag.from_delayed(partitions)

In [11]:
chk_rules_result_bd = cull_empty_partitions(chk_rules_result_bd)

In [12]:
# replace all the '\n' with empty string so that we can access it using xml.etree.ElementTree
chk_rules_result_bd = chk_rules_result_bd.map(lambda line: re.sub(r'(\n$)', '@@anyi@@', line))
#chk_rules_result_bd = chk_rules_result_bd.map(lambda line: re.sub(r'(\r\n$)', "\n<file name=''>\n<error line='' severity='',message='' source=''/>\n</file>", line))

## Start a Dask cluster using SLURM jobs as workers.

There are a couple of things we need to configure here:

disabling the mechanism to write on disk when workers run out of memory,
memory, CPUs, maximum time and number of workers per SLURM job,
dask folders for log files and workers data.
We recommend putting the log folder and workers data folders in your /nesi/nobackup/<project_code> folder, most indicated for temporary files (see NeSI File Systems and Quotas).

All of these options can be set in configuration files, see Dask configuration and Dask jobqueue configuration for more information.

In [13]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-30:30",# walltime="0-00:30",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

In [14]:
#cluster.adapt(minimum_jobs=20, maximum_jobs=200)
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.166:41803 Dashboard: http://192.168.94.166:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [15]:
######START######

In [16]:
#convert from bags to dataframe
df = chk_rules_result_bd.to_dataframe()

In [17]:
df.head()

,0
0,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
1,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
2,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
3,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
4,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."


In [18]:
df.tail()

,0
1260,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
1261,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
1262,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
1263,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
1264,@@anyi@@@@anyi@@


In [19]:
# Remove lines that contains only new line
#df = df[df[0]!= '@@anyi@@']
df = df[~df[0].str.contains('@@anyi@@')]

In [20]:
df.head()

,0
0,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
1,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
2,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
3,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
4,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."


In [21]:
df.tail()

,0
1259,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
1260,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
1261,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
1262,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."
1263,"\n<file name=""/scale_wlg_nobackup/filesets/nob..."


In [22]:
# convert back to dask bags
chk_rules_result_bd = bd.from_sequence(df.squeeze())

In [23]:
chk_rules_result_bd.take(1)

('\n<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/CheckStyle1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_30381834_6434168_4179_3.java">\n<error line="1" severity="error" message="File does not end with a newline." source="com.puppycrawl.tools.checkstyle.checks.NewlineAtEndOfFileCheck"/>\n<error line="1" severity="error" message="Missing package-info.java file." source="com.puppycrawl.tools.checkstyle.checks.javadoc.JavadocPackageCheck"/>\n<error line="1" column="1" severity="error" message="Utility classes should not have a public or default constructor." source="com.puppycrawl.tools.checkstyle.checks.design.HideUtilityClassConstructorCheck"/>\n<error line="1" column="14" severity="error" message="Name &apos;Code_30381834_6434168_4179_3&apos; must match pattern &apos;^[A-Z][a-zA-Z0-9]*$&apos;." source="com.puppycrawl.tools.checkstyle.checks.naming.TypeNameCheck"/>

In [24]:
######STOP######

In [25]:
# This will give us just the attributes in the file tag: <file ...>
#chk_rules_result_bd = chk_rules_result_bd.map(lambda row: html.unescape(row))

In [26]:
#replace new lines with empty strings
#df = df.replace(r'\n','', regex=True)

### Use ElementTree to get all the attrbutes and text in xmlor html tags

In [27]:
# This will give us just the attributes in the file tag: <file ...>
files_rows_bd_test1 = chk_rules_result_bd.map(lambda row: et.fromstring(row).attrib)

In [28]:
files_rows_bd_test1.take(1)

({'name': '/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/CheckStyle1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_30381834_6434168_4179_3.java'},)

In [29]:
import ntpath

In [30]:
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [31]:
def get_dict(dictionary):
    new_value = path_leaf(dictionary.get('name'))
    dictionary['name'] = new_value
    return dictionary

In [32]:
#@@#
# This will give us just the attributes in the file tag: <file ...> and extract the file name without the path
files_rows_bd_test2 = chk_rules_result_bd.map(lambda row: get_dict(et.fromstring(row).attrib))

In [33]:
#@@#
files_rows_bd_test2.take(1)

({'name': 'Code_30381834_6434168_4179_3.java'},)

def content(root):
    #return root.attrib
    #return root.tag
    for child in root:
        return child.tag, child.attrib


In [34]:
#@@#
def content(root):
    
    
        
   
    all_array =[]
    try:
        holder_dict = get_dict(root.attrib)
        for child in root:
            # return child.tag : child.attrib
            #holder_dict.append(child.attrib)
            all_array.append(holder_dict|child.attrib)
    except ParseError:
         all_array.append({
             'name': '',
             'line': '',
             'severity': '',
             'message': '',
             'source': ''
         })
        
    return all_array

In [35]:
#@@#
# This will give us just the attributes in the file tag: <file ...> and extract the file name without the path
files_rows_bd = chk_rules_result_bd.map(lambda row: content(et.fromstring(row)))

In [36]:
files_rows_bd = chk_rules_result_bd.map(lambda row: content(et.fromstring(row)))

In [37]:
#@@#
files_rows_bd.take(5)
##files_rows_bd.take(40065)

([{'name': 'Code_30381834_6434168_4179_3.java',
   'line': '1',
   'severity': 'error',
   'message': 'File does not end with a newline.',
   'source': 'com.puppycrawl.tools.checkstyle.checks.NewlineAtEndOfFileCheck'},
  {'name': 'Code_30381834_6434168_4179_3.java',
   'line': '1',
   'severity': 'error',
   'message': 'Missing package-info.java file.',
   'source': 'com.puppycrawl.tools.checkstyle.checks.javadoc.JavadocPackageCheck'},
  {'name': 'Code_30381834_6434168_4179_3.java',
   'line': '1',
   'column': '1',
   'severity': 'error',
   'message': 'Utility classes should not have a public or default constructor.',
   'source': 'com.puppycrawl.tools.checkstyle.checks.design.HideUtilityClassConstructorCheck'},
  {'name': 'Code_30381834_6434168_4179_3.java',
   'line': '1',
   'column': '14',
   'severity': 'error',
   'message': "Name 'Code_30381834_6434168_4179_3' must match pattern '^[A-Z][a-zA-Z0-9]*$'.",
   'source': 'com.puppycrawl.tools.checkstyle.checks.naming.TypeNameCheck'

## Flatten

In [38]:
files_rows_bd = files_rows_bd.flatten()

## Convert Bags to Dataframes

In [39]:
df = files_rows_bd.to_dataframe()
#df = files_rows_bd.to_dataframe(meta={'name':'str', 'line':'int', 'severity':'str', 'message':'str', 'source':'str'})

In [40]:
df

,name,line,severity,message,source
npartitions=101,,,,,
,object,object,object,object,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [41]:
df.columns

Index(['name', 'line', 'severity', 'message', 'source'], dtype='object')

In [42]:
df.dtypes

name        object
line        object
severity    object
message     object
source      object
dtype: object

In [43]:
df.npartitions

101

In [44]:
#df.head().compute()
df.head()

,name,line,severity,message,source
0,Code_30381834_6434168_4179_3.java,1,error,File does not end with a newline.,com.puppycrawl.tools.checkstyle.checks.Newline...
1,Code_30381834_6434168_4179_3.java,1,error,Missing package-info.java file.,com.puppycrawl.tools.checkstyle.checks.javadoc...
2,Code_30381834_6434168_4179_3.java,1,error,Utility classes should not have a public or de...,com.puppycrawl.tools.checkstyle.checks.design....
3,Code_30381834_6434168_4179_3.java,1,error,Name 'Code_30381834_6434168_4179_3' must match...,com.puppycrawl.tools.checkstyle.checks.naming....
4,Code_30381834_6434168_4179_3.java,2,error,Missing a Javadoc comment.,com.puppycrawl.tools.checkstyle.checks.javadoc...


In [45]:
df.shape[0].compute()

8074616

## Save all the codes from the posts into a CSV file